In [413]:
from openai import OpenAI
from docx import Document
from docx.shared import Pt, Inches, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml import OxmlElement, parse_xml
from docx.oxml.ns import qn
from datetime import date
import re


In [414]:
# Set up OpenAI API key
api_key = ''
client = OpenAI(api_key=api_key)

In [415]:
def generate_index(project_requirements):
    prompt = f"""
    Based on the following project requirements, generate a structured index for a Statement of Work document.
    The index should be in the format:
    1. Main topic
        a. Subtopic
            1. Sub-subtopic

    Project requirements: {project_requirements}

    Generate the index:
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates structured indexes for Statement of Work documents."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [416]:
def generate_content_for_topic(topic):
    prompt = f"""
    Generate detailed content for the following topic in a Statement of Work document:

    {topic}

    If this topic involves any kind of form or user input fields, include a table structure for it at the end of the content.
    The table should have the following columns: User Input, Input Type, Options, Comments.
    Provide at least 5 rows of realistic form fields if applicable.

    Start the table with '[[FORM_TABLE_START]]' and end it with '[[FORM_TABLE_END]]'.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates detailed content for Statement of Work documents, including form structures when necessary."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [417]:
def parse_index(index_content):
    lines = index_content.strip().split('\n')
    toc = []
    for line in lines:
        if line.strip():
            level = 1
            if line.startswith('    '):
                level = 3
            elif line.startswith('  '):
                level = 2
            toc.append((line.strip(), level))
    return toc

In [418]:
def create_custom_style(document, style_name, font_name, font_size, font_color, bold=False, italic=False):
    styles = document.styles
    custom_style = styles.add_style(style_name, WD_STYLE_TYPE.PARAGRAPH)
    font = custom_style.font
    font.name = font_name
    font.size = Pt(font_size)
    font.color.rgb = RGBColor(*font_color)
    font.bold = bold
    font.italic = italic
    return custom_style

In [419]:
def create_custom_styles(doc):
    create_custom_style(doc, 'Title Style', 'Arial', 24, (0, 0, 139), bold=True)  # Dark Blue
    create_custom_style(doc, 'Heading1 Style', 'Arial', 18, (0, 0, 0), bold=True)
    create_custom_style(doc, 'Heading2 Style', 'Arial', 16, (68, 68, 68), bold=True)
    create_custom_style(doc, 'Heading3 Style', 'Arial', 14, (102, 102, 102), bold=True)

In [420]:
def setup_header(section):
    header = section.header
    header_para = header.paragraphs[0]
    header_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    current_date = date.today().strftime("%d/%m/%Y")
    date_run = header_para.add_run(f'Date: {current_date}')
    date_run.font.color.rgb = RGBColor(183, 183, 183)
    date_run.font.name = 'Arial'
    date_run.font.size = Pt(10)

In [421]:
def add_page_number(run):
    fldChar = OxmlElement('w:fldChar')
    fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText')
    instrText.text = "PAGE"
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'end')
    
    run._element.append(fldChar)
    run._element.append(instrText)
    run._element.append(fldChar2)

In [422]:
def add_page_number_to_footer(footer):
    page_num_para = footer.add_paragraph()
    page_num_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    page_num_run = page_num_para.add_run('Page ')
    page_num_run.font.color.rgb = RGBColor(183, 183, 183)
    page_num_run.font.name = 'Arial'
    page_num_run.font.size = Pt(10)
    add_page_number(page_num_run)

In [423]:
def add_company_name_to_footer(footer, company_name):
    company_para = footer.add_paragraph()
    company_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    company_run = company_para.add_run(company_name)
    company_run.font.color.rgb = RGBColor(183, 183, 183)
    company_run.font.name = 'Arial'
    company_run.font.size = Pt(10)

In [424]:
def setup_footer(section, company_name):
    footer = section.footer
    add_page_number_to_footer(footer)
    add_company_name_to_footer(footer, company_name)

In [425]:
def setup_header_and_footer(doc, company_name):
    section = doc.sections[0]
    setup_header(section)
    setup_footer(section, company_name)

In [426]:
def create_title_page(doc, project_name, version, company_name):
    # Add a paragraph for the top border
    border_para = doc.add_paragraph()
    add_bottom_border(border_para)

    # Title
    title = doc.add_paragraph('Business Requirements Specifications', style='Title Style')
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Add a paragraph for the bottom border
    border_para = doc.add_paragraph()
    add_top_border(border_para)
    
    # Project details
    details = [
        f"Project Name: {project_name}",
        f"Version: {version}",
        # f"Date: {date.today().strftime('%d/%m/%Y')}",
        # f"Company: {company_name}"
    ]

    for detail in details:
        p = doc.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = p.add_run(detail)
        run.font.size = Pt(16)
        run.font.bold = True
        run.font.color.rgb = RGBColor(0, 0, 0)  # Black color

    doc.add_page_break()

def add_bottom_border(paragraph):
    p = paragraph._p  # Get the paragraph XML element
    pPr = p.get_or_add_pPr()  # Get or create the paragraph properties element
    pBdr = OxmlElement('w:pBdr')  # Create the paragraph border element
    bottom = OxmlElement('w:bottom')  # Create the bottom border element
    bottom.set(qn('w:val'), 'single')  # Set the border style
    bottom.set(qn('w:sz'), '24')  # Set the border width
    bottom.set(qn('w:space'), '1')  # Set the space attribute
    bottom.set(qn('w:color'), '000000')  # Set the border color to black
    pBdr.append(bottom)
    pPr.append(pBdr)

def add_top_border(paragraph):
    p = paragraph._p  # Get the paragraph XML element
    pPr = p.get_or_add_pPr()  # Get or create the paragraph properties element
    pBdr = OxmlElement('w:pBdr')  # Create the paragraph border element
    top = OxmlElement('w:top')  # Create the top border element
    top.set(qn('w:val'), 'single')  # Set the border style
    top.set(qn('w:sz'), '24')  # Set the border width
    top.set(qn('w:space'), '1')  # Set the space attribute
    top.set(qn('w:color'), '000000')  # Set the border color to black
    pBdr.append(top)
    pPr.append(pBdr)

In [427]:
def create_table_of_contents(doc, toc):
    doc.add_paragraph('Table of Contents', style='Heading1 Style')
    for item, level in toc:
        p = doc.add_paragraph()
        p.paragraph_format.left_indent = Pt((level - 1) * 12)  # Adjust indentation
        style = f'Heading{level} Style' if level <= 3 else 'Heading3 Style'
        p.style = doc.styles[style]
        p.add_run(item)
    doc.add_page_break()

In [428]:
def generate_form_table(form_name):
    prompt = f"""
    Generate a table structure for a {form_name} form. The table must have the following columns:
    1. User Input
    2. Input Type (e.g., checkbox, textfield, button) | '-'
    3. Options (if applicable) | '-'
    4. Comments | '-'

    Provide at least 5 rows of realistic form fields for a {form_name} form.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates form structures for software applications."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [429]:
def create_form_table(doc, table_content):
    rows = table_content.strip().split('\n')
    if len(rows) < 2:  # Ensure we have at least a header row and one data row
        doc.add_paragraph("Error: Invalid table data generated.")
        return

    # Determine the number of columns based on the first row
    first_row = rows[0].split('|')
    num_cols = len([cell for cell in first_row if cell.strip()])

    table = doc.add_table(rows=len(rows), cols=num_cols)
    table.style = 'Table Grid'
    
    for i, row in enumerate(rows):
        cells = row.split('|')
        for j, cell in enumerate([c for c in cells if c.strip()]):
            if j < num_cols:  # Ensure we don't exceed the number of columns
                table.cell(i, j).text = cell.strip()
    
    # Format header row
    for cell in table.rows[0].cells:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.bold = True
                # set_paragraph_shading(run, 204, 204, 204)
    
    # Add space after the table
    doc.add_paragraph().paragraph_format.space_after = Pt(12)

In [430]:
def set_paragraph_shading(paragraph, r, g, b):
    shading_elm = OxmlElement('w:shd')
    shading_elm.set(qn('w:val'), 'clear')
    shading_elm.set(qn('w:color'), 'auto')
    shading_elm.set(qn('w:fill'), f'{r:02x}{g:02x}{b:02x}')
    paragraph._p.get_or_add_pPr().append(shading_elm)

In [431]:
def apply_markdown_style(paragraph, text):
    # Bold
    bold_pattern = r'\*\*(.*?)\*\*'
    # Italic
    italic_pattern = r'\*(.*?)\*'
    # Headers
    header_pattern = r'^(#{1,6})\s(.+)$'

    # Check for header
    header_match = re.match(header_pattern, text)
    if header_match:
        level = len(header_match.group(1))
        text = header_match.group(2)
        paragraph.style = f'Heading{level}'
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        run = paragraph.add_run(text)
        run.bold = True
        font_size = 18 - (level * 2)  # Decrease font size for deeper levels
        run.font.size = Pt(font_size)
        return

    # Process bold and italic
    while '**' in text or '*' in text:
        bold_match = re.search(bold_pattern, text)
        italic_match = re.search(italic_pattern, text)
        
        if bold_match and (not italic_match or bold_match.start() < italic_match.start()):
            start, end = bold_match.span()
            paragraph.add_run(text[:start])
            run = paragraph.add_run(bold_match.group(1))
            run.bold = True
            text = text[end:]
        elif italic_match:
            start, end = italic_match.span()
            paragraph.add_run(text[:start])
            run = paragraph.add_run(italic_match.group(1))
            run.italic = True
            text = text[end:]
    
    # Add any remaining text
    if text:
        paragraph.add_run(text)

In [432]:
def create_content_pages(doc, toc):
    for item, level in toc:
        style = f'Heading{level} Style' if level <= 3 else 'Heading3 Style'
        p = doc.add_paragraph(style=style)
        p.add_run(item)
        set_paragraph_shading(p, 204, 204, 204)  # Set background color to RGB(204, 204, 204)
        p.paragraph_format.space_after = Pt(12)  # Add some space after the heading
        
        content = generate_content_for_topic(item)
        
        # Check if there's a form table in the content
        table_matches = list(re.finditer(r'\[\[FORM_TABLE_START\]\](.*?)\[\[FORM_TABLE_END\]\]', content, re.DOTALL))
        
        if table_matches:
            # Process content with tables
            last_end = 0
            for match in table_matches:
                # Add text content before the table
                text_before_table = content[last_end:match.start()]
                add_parsed_content(doc, text_before_table)
                
                # Add form table
                doc.add_paragraph("Form Structure:")
                create_form_table(doc, match.group(1).strip())
                
                # Add space after the table
                doc.add_paragraph().paragraph_format.space_after = Pt(12)
                
                last_end = match.end()
            
            # Add any remaining content after the last table
            remaining_content = content[last_end:]
            add_parsed_content(doc, remaining_content)
        else:
            # If no table, just add the content as is with Markdown parsing
            add_parsed_content(doc, content)
        
        if level == 1:
            doc.add_page_break()

def add_parsed_content(doc, content):
    for line in content.split('\n'):
        if line.strip():
            p = doc.add_paragraph()
            apply_markdown_style(p, line)

In [433]:
def set_page_margins(doc, left=0.5, right=0.5, top=1, bottom=1):
    sections = doc.sections
    for section in sections:
        section.left_margin = Inches(left)
        section.right_margin = Inches(right)
        section.top_margin = Inches(top)
        section.bottom_margin = Inches(bottom)

In [434]:
def set_default_font(document, font_name='Arial'):
    # Set the default font for the entire document
    font = document.styles['Normal'].font
    font.name = font_name

In [435]:
def create_sow_document(project_name, version, toc, company_name):
    doc = Document()
    set_page_margins(doc)  # Add this line to set the margins
    set_default_font(doc, 'Arial')
    create_custom_styles(doc)
    setup_header_and_footer(doc, company_name)
    create_title_page(doc, project_name, version, company_name)
    create_table_of_contents(doc, toc)
    create_content_pages(doc, toc)
    doc.save('sow_document.docx')

In [436]:
def main():
    project_name = "Oxford Mtrain"
    version = "1.0.0"
    company_name = "© Techuz Infoweb Pvt. Ltd."
    project_requirements = """The objective of this document is to define the features of an Oxford Mtrain Web Application development, which aims to develop a training platform for the medical therapist. Its primary objective is to provide a user-friendly and secure interface for therapists to get the training using a chatbot. Platform offers a meticulously designed user journey aimed at providing a seamless and enriching experience. Beginning with a straightforward Login/Signup process, users embark on a path that not only involves onboarding to complete their profiles and email verification but also necessitates the thoughtful purchase of subscription plans tailored to their training needs. As users delve into the system, they can access and practice a curated list of exercises, engaging in role plays with our AI chatbot patients. The platform's versatility allows users to switch roles between client and supervisor, fostering an intuitive and dynamic learning environment. The ability to switch between voice and text communication adds another layer of adaptability. Moreover, users can meticulously track their progress, receive scores from ChatGPT supervisors, and contribute valuable feedback to enhance the app's efficacy. With features like exporting conversations, seamless roleplays with ChatGPT supervisors, and robust profile management, our platform endeavors to provide therapists with a comprehensive and empowering journey for skill development and continuous improvement."""

    print("Generating index...")
    index_content = generate_index(project_requirements)
    toc = parse_index(index_content)
    
    print("Creating document and generating content...")
    create_sow_document(project_name, version, toc, company_name)
    print("SOW document has been generated and saved as 'sow_document.docx'")

In [437]:
main()

Generating index...
Creating document and generating content...


/home/kamlesh/.local/lib/python3.10/site-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


SOW document has been generated and saved as 'sow_document.docx'
